# Exemple d'utilisation du package

## Importation du package et Lecture du fichier de données

In [1]:
# import nécessaire au fonctionnement du package
from mypackage import Strategy_Manager, Strategy, strategy, DataFileReader

# import supplémentaire pour les stratégies utilisées comme exemple
import pandas as pd
import warnings
import plotly.io as pio
pio.renderers.default = "browser"

# Désactiver tous les avertissements
warnings.filterwarnings('ignore')

In [2]:
# Chemin vers le fichier de données sous format .csv ou .parquet
filepath = 'fichier_donnée.csv' 
# filepath = 'data.parquet'
# filepath = 'data2.parquet'

# Initialiser le lecteur de fichiers en indiquant le format des dates
reader = DataFileReader(date_format='%d/%m/%Y') 

# Lecture du fichier
data = reader.read_file(filepath, date_column='Date_Price')
data

,AAVEUSDT,ACMUSDT,ADAUSDT,ADXUSDT,AGLDUSDT,ALGOUSDT,ALICEUSDT,ALPACAUSDT,ALPHAUSDT,AMPUSDT,...,BATUSDT,BCHUSDT,BEAMUSDT,BELUSDT,BETAUSDT,BICOUSDT,BLZUSDT,BNBUPUSDT,BNBUSDT,BNTUSDT
Date_Price,,,,,,,,,,,,,,,,,,,,,
2022-01-01,266.4,5.211,1.3800,0.5729,1.705,1.7414,13.380,0.6264,0.6995,0.04964,...,1.2560,444.8,0.5862,1.651,0.64700,4.480,0.2208,543.00,527.3,3.345
2022-01-02,265.3,5.195,1.3770,0.5799,1.828,1.6734,13.060,0.6085,0.6978,0.04897,...,1.2999,447.7,0.6011,1.667,0.68177,4.223,0.2258,552.62,531.0,3.396
2022-01-03,267.2,5.152,1.3190,0.5798,1.718,1.7949,12.640,0.6186,0.7019,0.05239,...,1.3017,434.7,0.5934,1.616,0.63450,3.892,0.2273,506.38,511.9,3.428
2022-01-04,252.1,5.107,1.3090,0.5613,1.637,1.6725,13.650,0.6163,0.6981,0.04929,...,1.2825,427.1,0.5864,1.646,0.63164,3.920,0.2219,494.10,506.9,3.401
2022-01-05,230.3,4.727,1.2300,0.5276,1.570,1.5554,12.230,0.5660,0.6276,0.04682,...,1.1764,399.4,0.5339,1.465,0.57229,3.553,0.2064,411.33,474.1,3.297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,56.0,2.346,0.2652,0.1212,0.270,0.1715,1.091,0.2529,0.0801,0.00318,...,0.1869,103.4,0.1167,0.405,0.07050,0.284,0.0534,28.24,244.3,0.343
2022-12-27,57.5,2.345,0.2603,0.1182,0.264,0.1701,1.093,0.2504,0.0786,0.00312,...,0.1788,101.7,0.1164,0.401,0.06771,0.274,0.0531,28.76,246.7,0.339
2022-12-28,53.8,2.288,0.2482,0.1143,0.259,0.1670,1.058,0.2477,0.0762,0.00309,...,0.1680,99.2,0.1107,0.386,0.06701,0.264,0.0523,28.27,244.5,0.329


## Création d'une stratégie basée sur la volatilité

In [4]:
# Création d'une stratégie par héritage

class VolatilityBasedStrategy(Strategy):
    """
    Stratégie basée sur la volatilité des actifs.
    
    Si la volatilité d'un actif dépasse un seuil donné, une position short est prise.
    Si la volatilité est inférieure au seuil, une position long est prise.
    """

    def __init__(self, assets, volatility_threshold=0.02, window_size=10, rebalancing_frequency='D', allocation_method='equal'):
        """
        Initialise la stratégie avec les paramètres donnés.

        :param assets: Liste des actifs à trader.
        :param volatility_threshold: Seuil pour décider entre position long/short.
        :param window_size: Fenêtre pour le calcul de la volatilité.
        :param rebalancing_frequency: Fréquence de rééquilibrage ('D', 'W', etc.).
        :param allocation_method: Méthode d'allocation ('equal' ou 'volatility_weighted').
        """
        super().__init__(rebalancing_frequency=rebalancing_frequency, assets=assets)
        self.volatility_threshold = volatility_threshold
        self.window_size = window_size
        self.volatility = None
        self.allocation_method = allocation_method

    def fit(self, data: pd.DataFrame) -> None:
        """
        Calcule la volatilité historique pour chaque actif sur une fenêtre donnée.

        :param data: Données historiques (prix des actifs).
        """
        self.volatility = {}
        for asset in self.assets:
            daily_returns = data[asset].pct_change()
            self.volatility[asset] = daily_returns.rolling(window=self.window_size).std()

    def get_position(self, historical_data: pd.DataFrame, current_position: dict) -> dict:
        """
        Détermine les positions basées sur la volatilité de chaque actif.

        :param historical_data: Données historiques des actifs.
        :param current_position: Positions actuelles.
        :return: Dictionnaire des positions pour chaque actif.
        """
        if self.volatility is None:
            raise ValueError("La méthode fit() doit être appelée avant get_position().")
        
        positions = {}
        active_assets = []
        
        # Déterminer les signaux pour chaque actif
        for asset in self.assets:
            current_vol = self.volatility[asset].iloc[-1] if asset in self.volatility else None
            
            if current_vol is None or pd.isna(current_vol):
                # Actif sans données valides
                positions[asset] = 0
            else:
                # Décider entre position short ou long
                if current_vol > self.volatility_threshold:
                    positions[asset] = -1  # Position short
                else:
                    positions[asset] = 1  # Position long
                active_assets.append(asset)
        
        # Ajuster les allocations selon la méthode choisie
        if active_assets:
            if self.allocation_method == 'equal':
                position_size = 1.0 / len(active_assets)
                for asset in active_assets:
                    positions[asset] *= position_size
            elif self.allocation_method == 'volatility_weighted':
                total_vol = sum(self.volatility[asset].iloc[-1] for asset in active_assets)
                total_vol = total_vol or 1  # Évite division par zéro
                for asset in active_assets:
                    weight = self.volatility[asset].iloc[-1] / total_vol
                    positions[asset] *= weight
                    
        return positions


## Mise en place des variables nécessaire aux backtests

In [5]:
# Création de la liste des actifs utilisé
all_asset = data.columns.to_list()

# Création des instances et exécution des backtests
vol_strat_monthly = VolatilityBasedStrategy(
    assets=all_asset,
    volatility_threshold=0.02,
    window_size=10,
    rebalancing_frequency='M',
    allocation_method='equal'
)


# Création des dictionnaires de coûts
transaction_cost = {'AAVEUSDT': 0.5, 'ACMUSDT': 0.2, 'ADAUSDT': 0.4, 'ADXUSDT': 0.6, 'AGLDUSDT': 0.7, 
                    'ALGOUSDT': 0.001, 'ALICEUSDT': 0.8, 'ALPACAUSDT': 2, 'ALPHAUSDT': 0.8, 'AMPUSDT': 0.051}
slippage_cost = {'AAVEUSDT': 0.53, 'ACMUSDT': 0.03, 'ADAUSDT': 0.33, 'ADXUSDT': 0.63, 'AGLDUSDT': 0.73, 
                 'ALGOUSDT': 0.001, 'ALICEUSDT': 0.8, 'ALPACAUSDT': 21, 'ALPHAUSDT': 0.8, 'AMPUSDT': 0.01}

# Création d'un dictionnaire partiel de coûts 
transaction_cost_partiel = {'AAVEUSDT': 0.5, 'ACMUSDT': 0.2,}

# Création d'un dictionnaire des différentes stratégies à comparer avec 
#   en clé, le nom choisi de la stratégie
#   en valeur, un tuple contenant la stratégie, les coûts de transactions, les coûts de slippage
dico_strat = {
    'vol_strat_monthly': (vol_strat_monthly, 0.002, 0.0005)
}

# Création de l'instance utile à la comparaison
manager = Strategy_Manager(data, dico_strat)

# Exécution des backtests
manager.run_backtests()

## Résultat du Backtest

In [6]:
# Affichage des statistiques pour chaque stratégie
manager.print_statistics()


Statistiques : 
                   total_return  annual_return  profit_factor  volatility  \
vol_strat_monthly          -0.0        -0.0003         0.9839      0.0035   

                   sharpe_ratio  max_drawdown  sortino_ratio  VaR_95%  \
vol_strat_monthly       -0.0967       -0.0009        -0.0759  -0.0003   

                   CVaR_95%  Profit/Loss_Ratio  num_trades  win_rate  
vol_strat_monthly   -0.0005             0.6559          36       0.6  


In [7]:
# Affichage des statistiques pour une stratégie précise
manager.print_statistics(strategy_name="vol_strat_monthly")


Statistiques de la strategie 'vol_strat_monthly':
                   total_return  annual_return  profit_factor  volatility  \
vol_strat_monthly          -0.0        -0.0003         0.9839      0.0035   

                   sharpe_ratio  max_drawdown  sortino_ratio  VaR_95%  \
vol_strat_monthly       -0.0967       -0.0009        -0.0759  -0.0003   

                   CVaR_95%  Profit/Loss_Ratio  num_trades  win_rate  
vol_strat_monthly   -0.0005             0.6559          36       0.6  


In [8]:
# Affichage des statistiques des différents actifs pour la stratégie précise
manager.print_statistics(strategy_name="vol_strat_monthly",detail=True)

# Affichage des statistiques des différents actifs pour toutes les stratégies 
# manager.print_statistics(detail=True)


Statistiques : vol_strat_monthly
             total_return  annual_return  profit_factor  volatility  \
AAVEUSDT           0.0024         0.0520         1.4444      0.0246   
ACMUSDT            0.0070         0.1588         3.6659      0.0270   
ADAUSDT           -0.0027        -0.0552         0.6192      0.0237   
ADXUSDT            0.0030         0.0651         1.7773      0.0208   
AGLDUSDT          -0.0021        -0.0441         0.6801      0.0218   
ALGOUSDT          -0.0003        -0.0059         0.9392      0.0203   
ALICEUSDT         -0.0037        -0.0757         0.6094      0.0317   
ALPACAUSDT        -0.0018        -0.0373         0.7073      0.0253   
ALPHAUSDT          0.0003         0.0066         1.0655      0.0237   
AMPUSDT           -0.0066        -0.1302         0.3040      0.0288   
ANKRUSDT          -0.0029        -0.0584         0.5526      0.0231   
ANTUSDT            0.0005         0.0103         1.0776      0.0258   
ARDRUSDT          -0.0007        -0.0137   

### Analyse Backtest

Analyse.................

## Graphique

In [9]:
# Choix du backend utilisé pour les graphiques
backend = 'plotly' # 'plotly' # 'matplotlib' # 'seaborn'

In [10]:
# Graphique des différentes stratégies avec possibilité d'inclusion de coûts 
manager.plot_all_strategies(backend=backend,include_costs=True)

In [11]:
# Graphique des différents actifs d'une stratégie précise avec possibilité d'inclusion de coûts 
manager.plot_strategy(strategy_name="vol_strat_monthly",backend=backend,include_costs=True)

In [12]:
# Histogramme de comparaison des différentes stratégies
manager.compare_strategies(backend=backend)

### I. *Analyse des graphiques*

#### **1. Rendements Cumulés avec Coûts Inclus**
- Les rendements cumulés montrent une divergence claire entre les stratégies. La majorité des courbes descendent progressivement, indiquant que les frais de transaction impactent négativement les performances globales.
- Certaines stratégies subissent des pertes extrêmes, comme celle avec la courbe verte, qui semble presque s'annuler avant la fin de la période.
- Une courbe violette reste stable tout au long de la période, suggérant qu’aucune transaction ou qu’une approche très prudente a été adoptée.

#### **2. Rendements Cumulés sans Coûts Inclus**
- En supprimant les coûts, on observe une amélioration notable des performances globales. 
- Les courbes affichent une moindre pente descendante, et certaines deviennent légèrement positives.
Cependant, une grande partie des stratégies montre encore une incapacité à générer des rendements constants, indiquant des faiblesses structurelles dans les approches de trading, même en l'absence de frais.

#### **3. Rendements Cumulés des Actifs Individuels (Avec Coûts)**
- Ce graphique met en évidence que les performances des actifs sont fortement divergentes. Plusieurs actifs subissent des pertes significatives tout au long de la période.
- Certaines courbes individuelles montrent une volatilité marquée, tandis que d'autres présentent une baisse constante. Cela pourrait refléter une mauvaise adaptation des stratégies aux caractéristiques spécifiques des actifs.

#### **4. Rendements Cumulés des Actifs Individuels (Sans Coûts)**
- Sans les coûts, les courbes individuelles montrent une stabilité accrue. Certains actifs commencent à générer des rendements positifs, suggérant que les frais opérationnels jouent un rôle clé dans la détérioration des performances.
- Une courbe particulière (en rose) dépasse légèrement le point d’équilibre, ce qui indique un actif avec un potentiel à exploiter dans un environnement sans coûts.

#### **5. Comparaison des Métriques**
- Ce graphique permet de comparer les différentes métriques pour chaque stratégie. Les points marquants incluent :
    - Nombre de Transactions : Les stratégies les plus actives enregistrent des coûts plus élevés et des rendements moindres, ce qui reflète une fréquence de trading non optimisée.
    - Sharpe Ratio et Profit Factor : Les valeurs sont globalement faibles, voire négatives, indiquant une gestion inefficace du risque et des pertes importantes par rapport aux gains.
Win Rate : La majorité des stratégies ont un taux de succès inférieur à 50 %, montrant une prédominance de trades non rentables.


### II. *Analyse détaillée de la VolatilityBasedStrategy*



La stratégie VolatilityBasedStrategy repose sur une approche simple mais efficace pour exploiter les variations de volatilité dans les actifs financiers. Cette stratégie identifie les actifs ayant une volatilité élevée et adopte une position vendeuse (short), tandis que les actifs avec une faible volatilité sont favorisés pour des positions longues (long).

Lors de son initialisation, la stratégie définit trois paramètres principaux :
1. **volatility_threshold** : un seuil permettant de différencier les actifs à forte volatilité de ceux à faible volatilité.
2. **window_size** : la taille de la fenêtre glissante utilisée pour calculer la volatilité.
3. **rebalancing_frequency** : la fréquence à laquelle les positions sont réajustées, fixée par défaut à une base quotidienne ('D').

La méthode fit est utilisée pour calculer les volatilités historiques. Elle s’appuie sur les rendements journaliers des prix des actifs pour déterminer la volatilité moyenne sur une fenêtre glissante définie par window_size. Cela prépare les données pour les décisions futures. Ensuite, la méthode get_position évalue la volatilité actuelle d’un actif pour déterminer la position à adopter. Si la volatilité dépasse le seuil défini (volatility_threshold), une position vendeuse est prise. Si elle est inférieure, la stratégie favorise une position acheteuse. Si la volatilité n’est pas calculable (valeur NaN), la position reste neutre.

Cette stratégie est particulièrement utile pour s’adapter aux conditions de marché caractérisées par des périodes de forte incertitude (volatilité élevée) ou de calme relatif (volatilité faible). En théorie, elle permet de minimiser les risques en évitant les actifs trop volatils tout en capitalisant sur des actifs plus stables.



#### **Performance globale de la stratégie vol_strat_monthly**

En observant les graphiques globaux des rendements cumulés, plusieurs observations ressortent :

1. *Avec coûts de transaction* :
   - La stratégie montre une performance proche de l’équilibre. Les rendements cumulés oscillent légèrement autour du point initial mais restent légèrement négatifs à la fin de la période.
   - Les coûts de transaction affectent considérablement les résultats globaux, réduisant les marges déjà faibles en raison d’une activité de trading modérée.

2. *Sans coûts de transaction* :
   - Les rendements sont meilleurs et la stratégie reste proche de l’équilibre, voire légèrement positive sur certaines périodes.
   - La capacité de la stratégie à s’adapter à des périodes de forte volatilité semble limitée, suggérant que le seuil et la fenêtre de calcul de la volatilité pourraient être ajustés pour des conditions de marché spécifiques.

3. *Comparaison avec d’autres stratégies* :
   - La stratégie basée sur la volatilité surpasse certaines stratégies de type moyenne mobile (ma_strat_weekly et monthly), qui affichent des pertes importantes.
   - Cependant, elle reste globalement en retrait par rapport aux performances des stratégies passives ou des stratégies de momentum qui prennent peu de risques.



#### **Analyse spécifique de la stratégie VolatilityBasedStrategy**

En isolant la stratégie basée sur la volatilité, son comportement sur différents actifs peut être examiné en détail :

1. *Avec coûts inclus* :
   - Les rendements cumulés montrent des pertes faibles mais constantes sur plusieurs actifs. Cela s’explique par la fréquence de réajustement des positions et les coûts associés aux transactions.
   - La volatilité calculée semble parfois mal ajustée à certains actifs, ce qui entraîne des décisions de trading défavorables.

2. *Sans coûts* :
   - Sans l’impact des coûts, les rendements sont proches de l’équilibre. Certains actifs affichent même des gains légers, suggérant que la stratégie est viable dans un environnement où les frais de transaction sont négligeables.
   - Cependant, les marges restent faibles, ce qui indique que l’approche nécessite une optimisation supplémentaire pour générer des gains significatifs.

3. *Métriques* :
   - *Profit Factor* : Légèrement inférieur à 1, indiquant que les pertes sont légèrement supérieures aux gains sur la période.
   - *Sharpe Ratio* : Négatif avec coûts, reflétant un ajustement défavorable du rendement par rapport au risque.
   - *Win Rate* : Proche de 50 %, mais insuffisant pour compenser les pertes générées par les trades non gagnants.

Dans l’ensemble, bien que certains actifs montrent des résultats prometteurs, la stratégie reste limitée dans sa capacité à générer des rendements significatifs, en partie à cause des coûts de transaction et d’une exploitation suboptimale des variations de volatilité.



#### **Statistiques globales**

- *Total Return* : Une légère perte de -0.0016, indiquant une stratégie qui reste proche de l’équilibre avant prise en compte des coûts.
- *Annual Return* : La perte annuelle est modeste, à -0.0332.
- *Profit Factor* : Avec une valeur de 0.7571, cela signifie que pour chaque dollar perdu, seulement 75 cents ont été gagnés.
- *Volatility* : Faible (0.0203), ce qui reflète une approche défensive.
- *Sharpe Ratio* : Négatif (-1.6406), montrant que la stratégie n’a pas bien ajusté ses rendements par rapport au risque pris.
- *Nombre de trades (num_trades)* : 10 trades au total, ce qui reste modéré.
- *Win Rate* : 50 %, ce qui est neutre, mais insuffisant pour compenser les pertes.

Ces résultats indiquent que, bien que la stratégie soit stable, elle manque de rendement significatif et est affectée par les coûts de transaction.



#### **Performance sur actifs individuels**

La stratégie est testée sur plusieurs actifs (AAVEUSDT, ACMUSDT, etc.), avec des résultats variables :

- *AAVEUSDT* : Montre des rendements proches de l’équilibre, mais souvent négatifs (-0.3741).
- *AMPUSDT* : Affiche une performance légèrement positive, avec un total_return de 0.0239, mais reste isolé.
- La *volatilité* des actifs testés est en moyenne faible, ce qui favorise la stabilité mais limite les opportunités de gains importants.

Dans l’ensemble, bien que certains actifs montrent des résultats prometteurs, la stratégie reste limitée dans sa capacité à générer des rendements significatifs, en partie à cause des coûts de transaction et d’une exploitation suboptimale des variations de volatilité.
